---
# Projeto 2 - Ciência dos Dados

Nome: Daniel Pucciariello

Nome: Stephanie Liu

A proposta desse trabalho é avaliar se um tweet sobre o tema "Pastel" foi feito por qual tipo de usuário: um usuário que gosta de Pastel e teve uma boa experiência decorrente do fato dele ter comido o alimento ou somente demonstra gostar desse, um usuário que desgoste de pastel ou teve uma má experiência ao se alimentar dele ou um usuário cujo emprego da palavra pastel tenha sido utilizada para expressar um sentimento que não se relacione com o alimento de forma direta (irrelevante).

---

___
# Classificador automático de sentimento
## Preparando o ambiente no jupyter:

In [130]:
%%capture

#Instalando o tweepy
!pip install tweepy
!pip install emoji
!pip install preprocessor
!pip install TextBlob
!pip install preprocessing
!pip install preprocessing.text
!pip install keyword_tokenize
!pip install remove_unbound_punct
!pip install remove_urls


In [131]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import emoji
import preprocessor 
from textblob import TextBlob
import string
import preprocessing
# from preprocessing.text import keyword_tokenize, remove_unbound_punct, remove_urls

In [132]:
import preprocessing.text as ptext
from preprocessing.text import keyword_tokenize, remove_unbound_punct, remove_urls

text_string = "important string at: http://example.com"

clean_string = ptext.preprocess_text(text_string, [
    remove_urls,
    remove_unbound_punct,
    keyword_tokenize
])

FileNotFoundError: [Errno 2] No such file or directory: '/anaconda3/lib/python3.7/site-packages/preprocessing/data/bnc_wiktionary_corpus.txt'

In [116]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    
    text_subbed = re.sub(pattern, ' ', text)
    text_subbed=text_subbed.replace("\n", " ")
    text_subbed=text_subbed.replace("…", " ")
    text_subbed=text_subbed.replace("@", " ")
    text_subbed=text_subbed.replace("rt", "")
    text_subbed=text_subbed.replace("https", "")
    
    return text_subbed
#tirar coisas com https

In [102]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
# with open('auth.pass') as fp:    
#     data = json.load(fp)

#Configurando a biblioteca. Não modificar
# auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
# auth.set_access_token(data['access_token'], data['access_token_secret'])

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

---

In [103]:
data_treinamento = pd.read_excel("Excel_pastel.xlsx",sheet_name="Treinamento")

In [104]:
irrelevantes_raw = " ".join(data_treinamento[data_treinamento.Classe<=1].Treinamento)

In [105]:
irrelevantes = cleanup(irrelevantes_raw)
#irrelevantes

In [106]:
tabela_relativa_ngostam_irrelevantes = pd.Series(irrelevantes.lower().split()).value_counts(True)
tabela_absoluta_ngostam_irrelevantes = pd.Series(irrelevantes.lower().split()).value_counts(False)


In [107]:
gostam_raw = " ".join(data_treinamento[data_treinamento.Classe==2].Treinamento)
ngostam_raw = " ".join(data_treinamento[data_treinamento.Classe<2].Treinamento)

In [108]:
tabela_relativa_gostam = pd.Series(gostam_raw.lower().split()).value_counts(True)
tabela_absoluta_gostam = pd.Series(gostam_raw.lower().split()).value_counts(False)

In [109]:
Multiplicando_tabela_naogostam = tabela_relativa_ngostam_irrelevantes*2

In [110]:
tudo=irrelevantes_raw + gostam_raw
tudo=cleanup(tudo)
tudo_absoluto=pd.Series(tudo.lower().split()).value_counts(False)

In [111]:
data_teste = pd.read_excel("Excel_pastel.xlsx",sheet_name="Teste")

In [112]:
tudo_raw = tudo.split()
tudo_sem_repeticao = set(tudo_raw)

#tudo_sem_repeticao

In [125]:
a = preprocessing.text.clean("O daniel é lalalala 😀")


AttributeError: module 'preprocessing' has no attribute 'text'

In [95]:
#data_teste_idx= data_teste.set_index("Teste")
# vamos ter a nossa string relevante e string de não relevante
# vamos somar os dois = total
# vamos dar o split = lista total
# vamos jogaar a lista total no set --> conjunto de palavras unicas
# ao temos o len do set  -- numero das palavras unicas

# ao inves da freq relativa 

#     gosta=((tabela_absoluta_gostam[i.split()]+1)/(count_palavras_gostam+d)).prod()
#     ngosta=((tabela_absoluta_ngostam_irrelevantes[i.split()]+1)/(count_palavras_gostam+d)).prod()
lista=[]
valor_a = []

#Variaveis gostam 
gostam_limpo = p.clean(gostam_raw)
# gostam_limpo = cleanup(gostam_raw)
palavras_gostam = gostam_limpo.split()
ngostam_limpo = cleanup(ngostam_raw)

#Variaveis que não gostam
palavras_ngostam = ngostam_limpo.split()
count_palavras_gostam = len(palavras_gostam)
count_palavras_ngostam = len(palavras_ngostam)

d = len(tudo_sem_repeticao)

#fazer if pra verificar se a palavra eh inedita ou nao; se for inedita, vai ser igual ao termo abaixo

n_existe = 1/(count_palavras_gostam+d)

#se der underflow, tem que fazer log 
for i in data_teste.Teste:
    gosta = 1
    ngosta = 1
    for x in i.split():
        prob_g = 0
        prob_n = 0
        if x not in tabela_absoluta_gostam:
            prob_g= 1/(count_palavras_gostam+d)
            
        elif x in tabela_absoluta_gostam:
            prob_g=((tabela_absoluta_gostam[x]+1)/(count_palavras_gostam+d))
            
        if x not in tabela_absoluta_ngostam_irrelevantes:
            prob_n= 1/(count_palavras_ngostam+d)
            
        elif x in tabela_absoluta_ngostam_irrelevantes:
            prob_n=((tabela_absoluta_ngostam_irrelevantes[x]+1)/(count_palavras_ngostam+d))
            
        gosta*=prob_g
        ngosta*=prob_n
    
    if gosta>ngosta:
        valor=2
    elif gosta<ngosta:
        valor=1
    a = tabela_absoluta_gostam[i.split()]+1
    valor_a.append(a)
    lista.append(valor)
#print(valor_a)

data_teste.Classe = lista
# data_teste
# data_teste.to_excel("Teste_FeitoProg.xlsx")
errou=len(data_teste.loc[(data_teste.Classe==2)&(data_teste.Modelo<=1)])
acuracia=1-(errou/(len(data_teste)))
acuracia

AttributeError: module 'preprocessor' has no attribute 'clean'